In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from scipy.stats import skew
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
# from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xg

In [ ]:
drama = pd.read_csv("/content/netflix_titles.csv")
drama

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


In [ ]:
drama.isnull().sum().sort_values(ascending=False).head()

director      2389
cast           718
country        507
date_added      10
rating           7
dtype: int64

In [ ]:
drama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [ ]:
drama_new = drama[drama['type']=='TV Show'].reset_index()
drama_new = drama_new.drop(columns = ['duration','country','date_added','release_year','show_id','type','index','listed_in'])

In [ ]:
drama_new.head()

,title,director,cast,rating,description
0,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",TV-MA,In a future where the elite inhabit an island ...
1,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",TV-MA,A genetics professor experiments with a treatm...
2,1983,NaN,"Robert Więckiewicz, Maciej Musiał, Michalina O...",TV-MA,"In this dark alt-history thriller, a naïve law..."
3,1994,Diego Enrique Osorno,NaN,TV-MA,Archival video and new interviews examine Mexi...
4,Feb-09,NaN,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",TV-14,"As a psychology professor faces Alzheimer's, h..."


In [ ]:
drama_new['director'] = drama_new['director'].fillna("")
drama_new['cast'] = drama_new['cast'].fillna("")
drama_new['about'] = drama_new['cast']+drama_new['director']+drama_new['description']

In [ ]:
drama_new.head()

,title,director,cast,rating,description,about
0,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",TV-MA,In a future where the elite inhabit an island ...,"João Miguel, Bianca Comparato, Michel Gomes, R..."
1,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",TV-MA,A genetics professor experiments with a treatm...,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan..."
2,1983,,"Robert Więckiewicz, Maciej Musiał, Michalina O...",TV-MA,"In this dark alt-history thriller, a naïve law...","Robert Więckiewicz, Maciej Musiał, Michalina O..."
3,1994,Diego Enrique Osorno,,TV-MA,Archival video and new interviews examine Mexi...,Diego Enrique OsornoArchival video and new int...
4,Feb-09,,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",TV-14,"As a psychology professor faces Alzheimer's, h...","Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3,max_features = None,analyzer = 'word',token_pattern = 'r\w{1,}', ngram_range = (1,3), stop_words = 'english')
drama_new['about'] = drama_new['about'].fillna("")
tfv_matrix_drama_new = tfv.fit_transform(drama_new['about'])

from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix_drama_new,tfv_matrix_drama_new)
sig[0]
indices = pd.Series(drama_new.index,index = drama_new['title']).drop_duplicates()
indices

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ral', 'rd', 'rds', 'ready', 'reafter', 'reas', 'reby', 'red', 'ree', 'refore', 'rehand', 'rein', 'reover', 'rest', 'reupon', 'rever', 'rhaps', 'ribe', 'ring', 'rious', 'rly', 'rmer', 'rmerly', 'rom', 'ront', 'ross', 'rough', 'roughout', 'round', 'rs', 'rself', 'rselves', 'rst', 'rt', 'rtheless', 'rther', 'rty', 'ru', 'rwards', 'rwise', 'ry', 'ryone', 'rything', 'rywhere'] not in stop_words.
  warnings.warn(


title
3%                          0
46                          1
1983                        2
1994                        3
Feb-09                      4
                         ... 
Zindagi Gulzar Hai       2405
Zoids Wild               2406
Zombie Dumb              2407
Zona Rosa                2408
Zumbo's Just Desserts    2409
Length: 2410, dtype: int64

In [ ]:
def give_recommendation(title,sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores,key = lambda x:x[1], reverse = True)
    sig_scores = sig_scores[1:11]
    drama_new_indices = [i[0] for i in sig_scores]
    return drama_new['title'].iloc[drama_new_indices]

In [ ]:
give_recommendation("Crash Landing on You")

201            The Hymn of Death
10                     Mr. Queen
49     Arthdal Chronicles Part 3
17                  Mr. Sunshine
131                      Dong Yi
24        Twenty-Five Twenty-One
91         My Love from the Star
57           A Business Proposal
105                       Kairos
242      Love Playlist: Season 4
Name: Name, dtype: object

In [ ]:
kdrama = pd.read_csv("/content/kdrama.csv")
kdrama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  250 non-null    object 
 1   Aired Date            250 non-null    object 
 2   Year of release       250 non-null    int64  
 3   Original Network      250 non-null    object 
 4   Aired On              250 non-null    object 
 5   Number of Episodes    250 non-null    int64  
 6   Duration              250 non-null    object 
 7   Content Rating        245 non-null    object 
 8   Rating                250 non-null    float64
 9   Synopsis              250 non-null    object 
 10  Genre                 250 non-null    object 
 11  Tags                  250 non-null    object 
 12  Director              249 non-null    object 
 13  Screenwriter          249 non-null    object 
 14  Cast                  250 non-null    object 
 15  Production companies  2

In [ ]:
kkdrama = kdrama[kdrama['type']=='TV Show'].reset_index()
kkdrama = kdrama.drop(columns = ['Duration','Aired Date','Aired On','Rank','Original Network'])

<ipython-input-26-d410fea5cf65>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kkdrama = kdrama[drama['type']=='TV Show'].reset_index()


In [ ]:
kkdrama['About'] = kkdrama['Cast']+kkdrama['Director']+kkdrama['Synopsis']+kkdrama['Production companies']+kkdrama['Screenwriter']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3,max_features = None,analyzer = 'word',token_pattern = 'r\w{1,}', ngram_range = (1,3), stop_words = 'english')
kkdrama['About'] = kkdrama['About'].fillna("")
tfv_matrix_kkdrama = tfv.fit_transform(kkdrama['About'])

from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix_kkdrama,tfv_matrix_kkdrama)
sig[0]
indices = pd.Series(kkdrama.index,index = kkdrama['Name']).drop_duplicates()
indices

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ral', 'rd', 'rds', 'ready', 'reafter', 'reas', 'reby', 'red', 'ree', 'refore', 'rehand', 'rein', 'reover', 'rest', 'reupon', 'rever', 'rhaps', 'ribe', 'ring', 'rious', 'rly', 'rmer', 'rmerly', 'rom', 'ront', 'ross', 'rough', 'roughout', 'round', 'rs', 'rself', 'rselves', 'rst', 'rt', 'rtheless', 'rther', 'rty', 'ru', 'rwards', 'rwise', 'ry', 'ryone', 'rything', 'rywhere'] not in stop_words.
  warnings.warn(


Name
Move to Heaven              0
Flower of Evil              1
Hospital Playlist           2
Hospital Playlist 2         3
My Mister                   4
                         ... 
Live Up to Your Name      245
Queen for Seven Days      246
Memory                    247
A Korean Odyssey          248
Voice 4: Judgment Hour    249
Length: 250, dtype: int64

In [ ]:
def give_recommendation(title,sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores,key = lambda x:x[1], reverse = True)
    sig_scores = sig_scores[1:11]
    kkdrama_indices = [i[0] for i in sig_scores]
    return kkdrama['Name'].iloc[kkdrama_indices]

In [ ]:
give_recommendation("Flower of Evil")

35           Taxi Driver
80      The King of Pigs
173      Come and Hug Me
178    The Beauty Inside
248     A Korean Odyssey
162     Her Private Life
8       Alchemy of Souls
83            Empress Ki
158            Pinocchio
153    The Crowned Clown
Name: Name, dtype: object